In [73]:
import numpy as np
from math import log, sqrt

In [93]:
def characteristic_size(datapts_, pct):
    '''
    Calculate characteristic size based on percent finer
    
    Input
    ------
    datapts_: nd array of grain size diameters and percent finers
    pct: percentage such that D_pct is the size such that pct percent
        of the sample is finer than D_pct (between 0 and 100)
        
    Output
    ------
    Dx (in millimiters), where x percent of the sample is finer than Dx
    '''

    pct_ = float(pct)
    
    loc_ = np.where(datapts['fractions'] >= pct_/100)[0][0]
    
    assert loc_>0, 'Percent finer appears to be less than 0!'

    D2_ = log(datapts_['diameters'][loc_],2)
    D1_ = log(datapts_['diameters'][loc_-1],2)
    f2_ = datapts_['fractions'][loc_]
    f1_ = datapts_['fractions'][loc_-1]

    psix_ = D1_ + ((D2_ - D1_)/(f2_ - f1_)) * (pct_/100 - f1_)
    
    return 2**psix



def grain_statistics(datapts):
    '''
    Statistical characteristics of grain size distributions
    
    Input
    -----
    datapts: nd array of grain size diameters and percent finers
    
    Output
    ------
    D_g: Geometric mean grain diameter in millimiter
    sigma_g: Standard deviation of grain size in millimiters
        Sediment is well sorted if sigma_g < 1.6
    '''

    psi_b = [log(i,2) for i in datapts['diameters']]

    dtype = [('psi', float), ('diameters', float), ('fractions', float)]
    centerpts = np.zeros((len(datapts)-1), dtype=dtype)       # create a structured array

    for i in range(len(datapts)-1):    
        centerpts['psi'][i] = (psi_b[i+1] + psi_b[i])/2
        centerpts['diameters'][i] = sqrt(datapts['diameters'][i] * datapts['diameters'][i+1])
        centerpts['fractions'][i] = datapts['fractions'][i+1]  - datapts['fractions'][i]

    psi_bar = sum(centerpts['psi'] * centerpts['fractions'])
    D_g = 2 ** psi_bar

    sigma = sqrt(sum((centerpts['psi'] - psi_bar)**2 * centerpts['fractions']))
    sigma_g = 2 ** sigma
    
    return D_g, sigma_g




# init
datapoints = [(4,100),(2,99),(1,97),(0.5,83.4),(0.25,42),(0.125,10),(0.062,3.2),(0.031,2)]

dtype = [('diameters', float), ('fractions', float)]
datapts = np.array(datapoints, dtype=dtype)       # create a structured array

# scale
if max(datapts['fractions']) > 1.:
    datapts['fractions'] = datapts['fractions'] / 100.

datapts = np.sort(datapts, order='diameters')       

psi_scale = np.array(range(-10,20), dtype = np.float)
psi_scale_mm = 2**psi_scale

# interpolate
if datapts['fractions'][0] > 0.0:
    # if smallest fraction is not zero, interpolate linearly to find the missing value
    D3 = log(datapts['diameters'][1],2)
    D2 = log(datapts['diameters'][0],2)
    f3 = datapts['fractions'][1]
    f2 = datapts['fractions'][0]
    newpsi = D3 + ((D2 - D3)/(f2 - f3))*(0-f3)
    first_entry = np.array((2**newpsi,0), dtype=dtype)
    datapts = np.hstack((first_entry,datapts))
    
if datapts['fractions'][-1] < 1.0:
    # if largest fraction is not zero, assume that the next larger psi scale grain size had a fraction of 1
    next_larger_size = psi_scale_mm[np.where(psi_scale_mm>datapts['diameters'][-1])[0][0]]
    last_entry = np.array([(next_larger_size,1.0)], dtype = dtype)
    datapts = np.hstack((datapts,last_entry))
    


In [99]:
def characteristic_size(pct):

    if type(pct) is int or type(pct) is float:
        pct = [pct]

    for i in pct:

        pct_ = float(i)

        loc_ = np.where(datapts['fractions'] >= pct_/100)[0][0]

        assert loc_>0, 'Percent finer appears to be less than 0!'

        D2_ = log(datapts['diameters'][loc_],2)
        D1_ = log(datapts['diameters'][loc_-1],2)
        f2_ = datapts['fractions'][loc_]
        f1_ = datapts['fractions'][loc_-1]

        psix_ = D1_ + ((D2_ - D1_)/(f2_ - f1_)) * (pct_/100 - f1_)
        D_x_ = 2**psix_

        print 'The characteristic size for', i, 'percent finer is:', D_x_, 'mm'

In [100]:
characteristic_size(50)

The characteristic size for 50 percent finer is: 0.285831478754 mm
